In [1]:
import numpy as np
from ml_models import OptimizationModel
def run_optimization():
    # Define random bounds for the variables
    bounds = [
        {'name': 'var1', 'type': 'continuous', 'domain': (0, 50)},
        {'name': 'var2', 'type': 'continuous', 'domain': (0, 50)},
        {'name': 'var3', 'type': 'continuous', 'domain': (0, 50)}
    ]

    # Define a random target value
    target_value = np.random.randint(1, 200)
    print(target_value)

    # Define random reagent information
    reagent_info = {'reagent1': 20, 'reagent2': 30, 'reagent3': 25, 'reagent4': 50, 'reagent5': 15}  # Assuming volumes in μL

    # Define fixed reagents (randomly chosen)
    fixed_reagents = [('reagent2', 30), ('reagent3', 25), ('reagent4', 50)]  # Example fixed reagents with volumes

    # Initialize the optimization model
    opt_model = OptimizationModel(bounds, target_value, reagent_info, fixed_reagents, initial_design_numdata=15, batch_size=3, max_iters=40)

    def simulate_chemistry(x):
        # Define coefficients to simulate more nuanced chemistry behavior
        a, b, c = 1.0, 0.1, 0.05
        
        # Simulate an outcome with more predictable behavior
        result = a * x[:, 0] + b * x[:, 1]**2 - c * x[:, 2]
        
        # Add a small, controlled amount of noise to mimic experimental variation
        #noise = np.random.normal(0, 1, size=x.shape[0])
        return (result)
    def calc_obj(x):
        return abs(target_value - x).reshape(-1, 1)

    # Generate initial design and simulate experiments to get initial data
    X_initial = opt_model.generate_initial_design()
    Y_initial = calc_obj(simulate_chemistry(X_initial))

    print(type(X_initial))
    print(type(Y_initial))

    #print("x init", X_initial, 'yinit',Y_initial.shape)
    opt_model.experiment_data['X'] = list(X_initial)
    opt_model.experiment_data['Y'] = list(Y_initial)
    # Initialize the optimizer with initial experimental data
    opt_model.initialize_optimizer(X_initial, Y_initial)

    print(opt_model.optimizer)
    print(opt_model.acquisition)

    # Learning loop
    number_of_iterations = 75  # Define the number of iterations for the learning loop
    for i in range(number_of_iterations):
        # Suggest next locations for experiments
        next_locations = opt_model.suggest_next_locations()
        print(next_locations)
        
        # Simulate experiments at the suggested locations
        Y_new = calc_obj(simulate_chemistry(next_locations))
        
        # Update the optimizer with the new experimental data
        opt_model.update_experiment_data(next_locations, Y_new)
        
        print(f"Iteration {i+1}, Suggested Locations: {next_locations}, Results: {Y_new}")
        print(opt_model.quit, opt_model.curr_iter)
        
        # Optional: Check if the optimizer has met the target or other stopping criteria
        if opt_model.quit:
            print("Stopping criteria met.")
            break
    # To get the best observed X values (parameters)
    X_best = opt_model.optimizer.X[np.argmin(opt_model.optimizer.Y)]

    # To get the best observed Y value (function value)
    Y_best = np.min(opt_model.optimizer.Y)
    return i, target_value, (X_best, Y_best)


In [2]:
test_info = {}
for test in range(10):
    num_iter, tar, best = run_optimization()
    test_info[test] = (num_iter, tar, best)


167
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
1
MPI
[[50. 50.  0.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 1, Suggested Locations: [[50. 50.  0.]], Results: [[133.]]
False 1
2
LCB
[[43.94161636 39.44127038 31.56202288]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 2, Suggested Locations: [[43.94161636 39.44127038 31.56202288]], Results: [[30.92489612]]
False 2
0
EI
[[50.         32.98206284 32.57649915]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 3, Suggested Locations: [[50.         32.98206284 32.57649915]], Results: [[9.84717805]]
False 3
1
MPI
[[50.         34.5397194  29.04719364]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 4, Suggested Locations: [[50.         34.5397194  29.04719364]], Results: [[0.84686196]]
True 4
Stopping criteria met.
144
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
1
MPI
[[44.10283677 32.59573724  9.04293358]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 1, Suggested Locatio

In [4]:
vals = test_info.values()
vals

dict_values([(3, 167, (array([50.        , 34.5397194 , 29.04719364]), 0.8468619576654248)), (20, 144, (array([44.49572812, 31.62211444, 14.14218403]), 0.21556891642231335)), (3, 45, (array([44.97319898,  0.        ,  0.        ]), 0.026801021016297)), (9, 112, (array([25.39983978, 29.73306925, 50.        ]), 0.6946195197822931)), (8, 38, (array([37.11888759,  0.        ,  0.        ]), 0.8811124053832344)), (14, 87, (array([50.        , 19.39775833, 32.06727177]), 0.9760607649495938)), (35, 198, (array([50.        , 38.62924912, 29.27641898]), 0.2419322093927576)), (37, 160, (array([42.34210211, 34.54797617, 31.05872951]), 0.14543141106645407)), (2, 49, (array([50.       ,  0.       , 25.5692417]), 0.278462085175903)), (3, 12, (array([14.08571011,  0.32416171, 23.88561328]), 0.9019375231056195))])

In [5]:
h = [val[0] for val in vals]
i = [val[1] for val in vals]
print(h)
print(i)

# Print average nuber of iterations to meet stopping criteria with 3 variable parameters and max iters of 40:
print(sum(h)/len(h))

[3, 20, 3, 9, 8, 14, 35, 37, 2, 3]
[167, 144, 45, 112, 38, 87, 198, 160, 49, 12]
